In [1]:
from pdf2image import convert_from_path
import pytesseract
import numpy as np 
import cv2 
from PIL import Image
import re

In [2]:

pages = convert_from_path(r"C:\Users\prath\OneDrive\Desktop\jupyter Lab\Medical Project\backend\Documents\patient_details\pd_1.pdf",poppler_path=r'C:\poppler-22.04.0\Library\bin')

In [3]:
pages

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1867x2000>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2000x1398>]

In [4]:
pages[0].show()

In [5]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
text = text = pytesseract.image_to_string(pages[0],lang='eng')
print(text)

47/12/2020

Patient Medical Record

Patient Information Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight
9264 Ash Dr 95
New York City, 10005 .
United States Height:
190
In Case of Emergency
m _ eee ee
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History

Chicken Pox (Varicella):

IMMUNE IMMUNE
Have you had the Hepatitis B vaccination?

No
List any Medical Problems (asthma, seizures, headaches):

Migraine




In [6]:
def preprocess_image(img):
    grey_scale = cv2.cvtColor(np.array(img),cv2.COLOR_BGR2GRAY)
    resized_image = cv2.resize(grey_scale,None,fx=1.5,fy =1.5,interpolation=cv2.INTER_LINEAR)
    processed_image =cv2.adaptiveThreshold(
    resized_image,255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY,63,13
    ) 
    return processed_image

In [7]:
img = preprocess_image(pages[0])
Image.fromarray(img).show()
text= pytesseract.image_to_string(img,lang='eng')
print(text)

17/12/2020

Patient Medical Record

Patient Information Birth Date

Kathy Crawford May 6 1972

(737) 988-0851 Weight’

9264 Ash Dr 95

New York City, 10005 ‘

United States Height
190

In Case of Emergency :
eee
Simeone Crawford 9266 Ash Dr
H New York City, New York, 10005
ome phone United States
(990) 375-4621
Work phone
Genera! Medical History
a

a em A IT a

ene

nr ene

Chicken Pox (Varicella): Measies:

IMMUNE IMMUNE

Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches):

Migraine



### Extract Name

In [8]:
pattern = "Patient Information(.*?)\(\d{3}\)"
matches = re.findall(pattern,text,flags = re.DOTALL) 
matches

[' Birth Date\n\nKathy Crawford May 6 1972\n\n']

In [9]:
matches[0]

' Birth Date\n\nKathy Crawford May 6 1972\n\n'

In [10]:
match = matches[0].replace("Birth Date","").strip()

In [11]:
pattern = "((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)"
date_matches = re.findall(pattern,match)
date = date_matches[0][0]
date

'May 6 1972'

In [12]:
match.replace(date,"").strip()

'Kathy Crawford'

In [13]:
def remove_noise_from_name(name):
    name = matches[0].replace("Birth Date","").strip()
    date_pattern = "((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)"
    date_matches = re.findall(pattern,match)
    
    if date_matches:
        date = date_matches[0][0]
        name = name.replace(date,"").strip()
    
    return name

In [14]:
name = " Birth Date\n\nKathy Crawford May 6 1972\n\n"
name = remove_noise_from_name(name)
name

'Kathy Crawford'

### Extract Phone

In [15]:
pattern = "Patient Information((.*?)(\(\d{3}\) \d{3}-\d{4}))"

matches = re.findall(pattern,text,flags = re.DOTALL)
matches

[(' Birth Date\n\nKathy Crawford May 6 1972\n\n(737) 988-0851',
  ' Birth Date\n\nKathy Crawford May 6 1972\n\n',
  '(737) 988-0851')]

In [16]:
matches[0][2]

'(737) 988-0851'

### Extract Vaccine 

In [17]:
pattern = "Have you had the Hepatitis B vaccination\?.*(Yes|No)"
matches = re.findall(pattern,text,flags = re.DOTALL)
matches

['No']

### Medical Problems

In [18]:
pattern = "List any Medical Problems (asthma, seizures, headaches):(.*)"
matches = re.findall(pattern,text,flags = re.DOTALL)
matches

[]